In [1]:
import sys
sys.path.insert(1, '/home/lruffati/Documents/0.University/0.SistemiElettorali/')
import src

from src.Metaclasses import *
import src.GlobalVars as GlobalVars
import random

In [2]:
class HubC:
    def __init__(self, *args):
        self.instances_dict = {}
        self.subs_relations = {}
    
    def get_subdivisions(self, sup, sub_name):
        """
        sup: o un'istanza di una geoEnt, o una tupla (nomeClasse, nomeIstanza)
        sub_name: nome classe target

        return: list of names (stringhe)
        """
        print(sup, sub_name)
        typ = None
        inst = sup
        if type(sup) == tuple:
            typ = sup[0]
            inst = self.get_instance(typ, sup[1])
        else:
            typ = sup.type
            name = sup.name
        
        var_name = self.subs_relations[typ][sub_name]
        
        print("getattr ", inst, var_name)
        var_deb = inst
        return getattr(inst, var_name)
    
    def get_superdivision(self, sub, sup_name):
        """
        sub: o un'istanza o una tuple
        sup_name: nome classe target

        return nome istanza target
        """
        pass

    def add_instance(self, class_name, inst_name, instance):
        instances = self.instances_dict.get(class_name, {})
        instances[inst_name] = instance
        self.instances_dict[class_name] = instances
    
    def get_instance(self, class_name, inst_name):
        print("Get inst: ", class_name, inst_name)
        return self.instances_dict[class_name][inst_name]

    def add_subdiv(self, sup_type, sub_type, var_name):
        """
        Records that instances of sup_type have instances of sub_type 
        stored in self.var_name
        """
        print("Adding sub: ",sup_type, sub_type, var_name)
        sup_dict = self.subs_relations.get(sup_type, {})
        sup_dict[sub_type] = var_name
        self.subs_relations[sup_type]= sup_dict
        
        

In [3]:
import pandas as pd
df1 = pd.DataFrame({'a':[1,2,3], 'b':[4,5,6]})
df2 = pd.DataFrame({'a':[11,12,13], 'b':[14,15,16]})
hub1 = HubC()
hub2 = HubC(1,2,3,4)

In [4]:
src.GlobalVars.Hub = hub1

In [5]:
class example:
    def __init__(self, val):
        #print("Instance ex")
        self.v = val
        self.id = random.randint(1,100)
        src.GlobalVars.Hub.add_instance("Plurinominale", self.id, self)
    
    def totals(self, *args, **kwargs):
        # print("Example: ",self.id)
        print("Funzione chiamata con: ", args, kwargs)
        # print("Funzione: ", self.v)
        return self.v

In [9]:
class cleanup(type):
    def __new__(mcs, *args, **kwargs):
        print("Conf cleanup: ", args, kwargs)
        if '__init__' in args[2]:
            o_init = args[2].get('__init__')
            def n__init__(self, name, **kwargs):
                print("Init cleanup preesistente")
                src.GlobalVars.Hub.add_instance(args[0], name, self)
                self.name = name
                return o_init(self, [], **kwargs)
            args[2]['__init__'] = n__init__
        else:
            def n_init__(self, name, **kwargs):
                print("init cleanup nuovo")
                self.name = name
                src.GlobalVars.Hub.add_instance(args[0], name, self)
            
            args[2]['__init__'] = n_init__
        
        args[2]['type'] = args[0]
        
        return super().__new__(mcs, *args)

metas = tuple([superdivision, totals, lanes, external, cleanup])
comb = type("combinata", metas, {})

In [11]:
conf = {'lane': {'plurinominale': {'first_input': 'kwarg',
                            'info_name': 'Circoscrizione',
                            'node_type': 'node',
                            'operations': [{'collect_type': 'lista',
                                            'corrector': 'commons.correct',
                                            'ideal_distribution': '$'}],
                            'sub_level': 'Plurinominale'}},
 'lanes_propose': {'elettori': {'distribution': ['Elettore', 'Seggi'],
                                'info': ['RestoCircoCoal',
                                         'RestoCircoCoalUsato'],
                                'source': {'args': [{'source': {'columns': ['Elettore -> Party',
                                                                            'Voti -> Votes'],
                                                                'name': 'self.eleggibili',
                                                                'type': 'fun'}},
                                                    {'source': {'name': 'self.subs_plurinominali_seggi',
                                                                'type': 'fun'}}],
                                           'columns': ['Party -> Elettore',
                                                       'Seats -> Seggi',
                                                       'Remainder -> RestoCircoCoal',
                                                       'RemainderUsed -> RestoCircoCoalUsato'],
                                           'name': 'commons.hondt',
                                           'type': 'fun'}},
                   'liste': {'distribution': ['Lista', 'Seggi'],
                             'info': ['RestoCircoLista',
                                      'RestoCircoListaUsato'],
                             'source': {'kwargs': {'seggi': {'source': {'name': 'constraints',
                                                                        'type': 'kwarg'}}},
                                        'name': 'self.ripartisci_seggi',
                                        'type': 'fun'}}},
 'metaclasses': ['superdivision', 'totals', 'lanes', 'external'],
 'subdivisions': {'plurinominali': {'functions': [{'name': 'seggi',
                                                   'source': {'name': 'self.seggi',
                                                              'type': 'att'}},
                                                  {'name': 'liste',
                                                   'source': {'totals': 'lista'}},
                                                  {'name': 'coalizioni',
                                                   'source': {'totals': 'coalizione'}}],
                                    'type': 'Plurinominale'}},
 'totals': {'coalizione': {'keys': ['Coalizione'],
                           'ops': {'Voti': 'sum'},
                           'source': {'name': 'self.subs_plurinominali_coalizioni',
                                      'type': 'fun'},
                           'type': 'aggregate'},
            'lista': {'keys': ['Lista'],
                      'ops': {'Voti': 'sum'},
                      'source': {'name': 'self.subs_plurinominali_liste',
                                 'type': 'fun'},
                      'type': 'aggregate'}},
 'totals_support': {'eleggibili': {'args': [{'source': {'args': ['coal_eletta'],
                                                        'columns': ['Coalizione -> Elettore',
                                                                    'Voti'],
                                                        'totals': 'coalizione'},
                                             'type': 'scalar'},
                                            {'source': {'args': ['eletta_sola'],
                                                        'columns': ['Lista -> Elettore',
                                                                    'Voti'],
                                                        'totals': 'lista'},
                                             'type': 'scalar'}],
                                   'function': 'commons.concat',
                                   'type': 'combine'},
                    'elettore_lista': {'ops': [{'column': 'Lista',
                                                'column_type': 'Partito',
                                                'replace_name': 'Elettore',
                                                'source': {'attr': ['elettore'],
                                                           'name': 'commons.exec_fun',
                                                           'type': 'fun'},
                                                'type': 'column'}],
                                       'source': {'totals': 'lista'},
                                       'type': 'transform'},
                    'ripartisci_seggi': {'args': [{'source': {'args': ['eletta'],
                                                              'rename': {'Lista': 'Party',
                                                                         'Voti': 'Votes'},
                                                              'totals': 'lista'},
                                                   'type': 'dataframe'},
                                                  {'source': {'columns': ['Lista -> Party',
                                                                          'Elettore'],
                                                              'name': 'self.elettore_lista',
                                                              'type': 'fun'},
                                                   'type': 'dataframe'},
                                                  {'source': {'columns': ['Elettore',
                                                                          'Seggi -> seats'],
                                                              'name': 'seggi',
                                                              'type': 'kwarg'},
                                                   'type': 'series'}],
                                         'function': 'commons.hondt',
                                         'keys': ['Elettore'],
                                         'merge_keys': ['Party'],
                                         'rename': {'Party': 'Lista',
                                                    'Remainder': 'RestoCircoLista',
                                                    'RemainderUsed': 'RestoCircoListaUsato',
                                                    'Seats': 'Seggi'},
                                         'type': 'combine'}}}

In [12]:
for mcs in metas:
    if hasattr(mcs, 'parse_conf'):
        conf = mcs.parse_conf(conf)

Functions before:  [{'name': 'seggi', 'source': {'name': 'self.seggi', 'type': 'att'}}, {'name': 'liste', 'source': {'totals': 'lista'}}, {'name': 'coalizioni', 'source': {'totals': 'coalizione'}}]
fun conf:  {'name': 'seggi', 'source': {'name': 'self.seggi', 'type': 'att'}}
fun conf:  {'name': 'liste', 'source': {'totals': 'lista'}}
fun conf:  {'name': 'coalizioni', 'source': {'totals': 'coalizione'}}
Functions after:  [{'name': 'seggi', 'source': {'source': {'name': 'self.seggi', 'type': 'att'}, 'type': 'fun'}}, {'name': 'liste', 'source': {'source': {'totals': 'lista'}, 'type': 'fun'}}, {'name': 'coalizioni', 'source': {'source': {'totals': 'coalizione'}, 'type': 'fun'}}]


In [13]:
conf

{'lane': {'plurinominale': {'first_input': 'kwarg',
   'info_name': 'Circoscrizione',
   'node_type': 'node',
   'operations': [{'collect_type': 'lista',
     'corrector': 'commons.correct',
     'ideal_distribution': '$'}],
   'sub_level': 'Plurinominale'}},
 'lanes_propose': {'elettori': {'distribution': ['Elettore', 'Seggi'],
   'info': ['RestoCircoCoal', 'RestoCircoCoalUsato'],
   'source': {'args': [{'source': {'columns': ['Elettore -> Party',
        'Voti -> Votes'],
       'name': 'self.eleggibili',
       'type': 'fun'}},
     {'source': {'name': 'self.subs_plurinominali_seggi', 'type': 'fun'}}],
    'columns': ['Party -> Elettore',
     'Seats -> Seggi',
     'Remainder -> RestoCircoCoal',
     'RemainderUsed -> RestoCircoCoalUsato'],
    'name': 'commons.hondt',
    'type': 'fun'}},
  'liste': {'distribution': ['Lista', 'Seggi'],
   'info': ['RestoCircoLista', 'RestoCircoListaUsato'],
   'source': {'kwargs': {'seggi': {'source': {'name': 'constraints',
       'type': 'kwarg'

In [ ]:
print("Expanded: ",conf)
confparse = src.Metaclasses.sources_parse.source_parse(conf)

In [ ]:
confparse

In [ ]:
class Circoscrizione(metaclass=comb, totals={},**confparse):
    def example_manual_fun(self, inp, out):
        print(inp)
        return out

In [ ]:
Circoscrizione = comb("Circoscrizione", (), {}, **confparse)

In [ ]:
a1 = example(df1)
a2 = example(df2)

In [ ]:
c = Circoscrizione("Ciao", plurinominali=[a1.id, a2.id])

In [ ]:
c.

In [ ]:
c.subs_liste()

In [ ]:
src.GlobalVars.Hub.get_subdivisions(("Circoscrizione", "Ciao"), "Plurinominale")